#Identification of Duplicate Question

Method:

- converting Questions into vectors using Word2vec
- using Siamese network to detect if the pair is duplicate


#Import Libraries

import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Get the data



In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/natural language processing/Identificarion of duplicate Questions/train.csv')
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
from sklearn.model_selection import train_test_split
X = data[['question1','question2']]
y = data['is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train['question1'] = X_train['question1'].apply(lambda x: (str(x)))
X_train['question2'] = X_train['question2'].apply(lambda x: (str(x)))


X_test['question1'] = X_test['question1'].apply(lambda x: (str(x)))
X_test['question2'] = X_test['question2'].apply(lambda x: (str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
question1 = list(X_train['question1'])
question2 = list(X_train['question2'])

test1=list(X_test['question1'])
test2=list(X_test['question2'])

- Tokenize the corpus, then transform sentence into sequences of integer corresponding to tokenizer word index

In [ ]:
tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(question1+question2+test1+test2)

In [ ]:
import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

question1_word_sequences = tokenizer.texts_to_sequences(question1)
question2_word_sequences = tokenizer.texts_to_sequences(question2)
word_index = tokenizer.word_index #unique words in corpus (training and test sets)

print("Words in index: %d" % len(word_index))

Words in index: 95596


In [ ]:

test1_word_sequences = tokenizer.texts_to_sequences(test1)
test2_word_sequences = tokenizer.texts_to_sequences(test2)

In [ ]:
(test1_word_sequences[0])

[4, 9, 5, 318, 1067, 120, 8, 1271]

In [ ]:
q1_data = pad_sequences(question1_word_sequences, maxlen=25)
q2_data = pad_sequences(question2_word_sequences, maxlen=25)
test1_data=pad_sequences(test1_word_sequences, maxlen=25)
test2_data=pad_sequences(test2_word_sequences, maxlen=25)

labels = np.array(y, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (270874, 25)
Shape of question2 data tensor: (270874, 25)
Shape of label tensor: (404290,)


In [ ]:
test1_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    4,    9,    5,  318, 1067,
        120,    8, 1271], dtype=int32)

- Word embedding dictionary

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/Colab Notebooks/natural language processing/Identificarion of duplicate Questions/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(95597, 100)

- Build the model

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GlobalAveragePooling1D,Lambda,Bidirectional
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras import backend as K

In [ ]:
def vec_distance(vects):
    x, y = vects
    return K.sum(K.square(x - y), axis=1, keepdims=True)

In [ ]:
def vec_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
from keras.layers.embeddings import Embedding

nb_words=embedding_matrix.shape[0]
max_sentence_len=25
embedding_layer = Embedding(nb_words,embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_sentence_len,trainable=False)

In [ ]:
lstm_layer =LSTM(128)

sequence_1_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

distance=Lambda(vec_distance, output_shape=vec_output_shape)([x1, y1])
dense1=Dense(16, activation='sigmoid')(distance)
dense1 = Dropout(0.3)(dense1)

bn2 = BatchNormalization()(dense1)
prediction=Dense(1, activation='sigmoid')(bn2)

model = Model([sequence_1_input, sequence_2_input], prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 100)      9559700     input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 128)          117248      embedding_1[4][0]            

In [ ]:
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])

In [ ]:
 np.stack((test1_data, test2_data), axis=1)[:,1]

(133416, 25)

In [ ]:
early_stopping =EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
X = np.stack((q1_data, q2_data), axis=1)
target = labels


Q1_train = np.stack((q1_data, q2_data), axis=1)[:,0]
Q2_train =np.stack((q1_data, q2_data), axis=1)[:,1]
Q1_val =  np.stack((test1_data, test2_data), axis=1)[:,0]
Q2_val =  np.stack((test1_data, test2_data), axis=1)[:,1]

In [ ]:

training =model.fit([Q1_train, Q2_train], y_train, validation_data=([Q1_val, Q2_val], y_test), verbose=1, epochs=5, batch_size=256, shuffle=True,class_weight=None, callbacks=[early_stopping])

Epoch 1/5
1059/1059 [==============================] - 369s 348ms/step - loss: 0.5402 - acc: 0.7258 - val_loss: 0.4804 - val_acc: 0.7668
Epoch 2/5
1059/1059 [==============================] - 463s 437ms/step - loss: 0.4653 - acc: 0.7777 - val_loss: 0.4454 - val_acc: 0.7924
Epoch 3/5
1059/1059 [==============================] - 468s 442ms/step - loss: 0.4339 - acc: 0.7989 - val_loss: 0.4191 - val_acc: 0.8056
Epoch 4/5
1059/1059 [==============================] - 468s 442ms/step - loss: 0.4112 - acc: 0.8131 - val_loss: 0.4094 - val_acc: 0.8109
Epoch 5/5
1059/1059 [==============================] - 438s 414ms/step - loss: 0.3938 - acc: 0.8240 - val_loss: 0.4019 - val_acc: 0.8149


In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open("brnn_model_distance_128_d16_d05.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("brnn_model_distance_128_d16_d05.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# load json and create model
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Colab Notebooks/natural language processing/Identificarion of duplicate Questions/brnn_model_distance_128_d16_d05.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights("/content/drive/MyDrive/Colab Notebooks/natural language processing/Identificarion of duplicate Questions/brnn_model_distance_128_d16_d05.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
pred=model.predict([test1_data, test2_data],verbose=1)

4170/4170 [==============================] - 46s 11ms/step


In [ ]:
Q1 = 'What is the meaning of healthy routine'
Q2 = 'What should we do to follow healthy routine'
# prediction = model.predict(X = np.stack((Q1,Q2), axis=1), verbose=1)

In [ ]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenize(Q1, Q2):
  que1 = tokenizer.texts_to_sequences(list(Q1.split()))
  que2 = tokenizer.texts_to_sequences(list(Q2.split()))
  qu1 =(np.array(que1)).flatten()
  qu2 =(np.array(que1)).flatten()
  q1 = np.pad(qu1, (25-len(qu1),0), 'constant')
  q2 = np.pad(qu2, (25-len(qu2),0), 'constant')
  return q1, q2



In [ ]:
que1 = tokenizer.texts_to_sequences(list(Q1.split()))
que2 = tokenizer.texts_to_sequences(list(Q2.split()))

In [ ]:
qu1 =(np.array(que1)).flatten()
qu2 =(np.array(que1)).flatten()

In [ ]:
q1 = np.pad(qu1, (25-len(qu1),0), 'constant')
q2 = np.pad(qu2, (25-len(qu2),0), 'constant')

In [ ]:
q1

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    2,    3,    1,  280,
         10,  700, 2631])

In [ ]:
predi=model.predict([q1, q2],verbose=1)

1/1 [==============================] - 1s 582ms/step


In [ ]:
p = np.mean(predi)

In [ ]:
(np.where(p > 0.5, 1, 0)).flatten()

array([1])

In [ ]:
y_pred =(np.where(pred > 0.5, 1, 0)).flatten()

In [ ]:
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
y_true = np.array(y_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.8148797745397853

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

confusion_matrix(y_true, y_pred)

array([[69003, 14972],
       [ 9726, 39715]])

In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.88      0.82      0.85     83975
     class 1       0.73      0.80      0.76     49441

    accuracy                           0.81    133416
   macro avg       0.80      0.81      0.81    133416
weighted avg       0.82      0.81      0.82    133416



In [ ]:
A = np.array([1,2,3,4,5])

len(np.pad(A, (25-len(A),0), 'constant'))

25